In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
import os

#Variables initialisation

src = 'data23.11'
out = src+'result'
table_name = 'data23.11/table.xlsx'
list_name = 'Лист1'
outtable_name = f"{out}/result_table.xlsx"
plot = True

out_table_column_width = 7

if not os.path.exists(out):
    os.makedirs(out)

In [2]:
wb = openpyxl.load_workbook(filename = table_name)
sheet = wb[list_name]
if os.path.exists(outtable_name):
    outwb = openpyxl.load_workbook(filename=outtable_name)
    outsheet = outwb['лист 1']
else:
    outwb = openpyxl.Workbook()
    outsheet = outwb.create_sheet('лист 1')

In [3]:
# рассчет сдвигов двумя способами
index = 0
i = 0
while i < 70:
    i+=1
    if sheet.cell(row=1, column=i).value == None:
        continue

    # основной код обработки столбика из таблицы
    row_s = 3
    row_end = row_s
    while sheet.cell(row=row_end, column=i).value is not None:
        row_end+=1
    # копируем исходные данные
    for t in range(2):
        for j in range(3, row_end):
            if type(sheet.cell(row=j, column=i+t).value) == str:
                outsheet.cell(row=j, column=1+index*out_table_column_width+t).value = (str(sheet.cell(row=j, column=i+t).value).strip())
            else:
                outsheet.cell(row=j, column=1+index*out_table_column_width+t).value = sheet.cell(row=j, column=i+t).value
        for j in range(1, 3):
            outsheet.cell(row=j, column=1+index*out_table_column_width+t).value = sheet.cell(row=j, column=i+t).value


    table = pd.DataFrame({
        'сила, кН':
          [i for i, in sheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=i, max_col=i,  values_only=True)],
        'file_name':
          ['WFM' + (str(i) if len(str(i)) > 1 else '0'+str(i))+'.csv' for i, in
               sheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=i+1, max_col=i+1,  values_only=True)]
    })

    deltasFFT = [0]
    deltasCorr = [0]
    base = np.loadtxt(os.path.join(src, table['file_name'][0]), skiprows=1, delimiter=',')
    base[:, 1] -= base[:, 1].mean()
    baseFFT = np.fft.rfft(base[:, 1])
    maxk = np.abs(baseFFT).argmax()
    if not os.path.exists(os.path.join(out, str(index))):
        os.makedirs(os.path.join(out, str(index)))
    for secondI in range(1, len(table['file_name'])):
        shifted = np.loadtxt(os.path.join(src, table['file_name'].iloc[secondI]), skiprows=1, delimiter=',')
        shifted[:, 1] -= shifted[:, 1].mean()
        shFFT = np.fft.rfft(shifted[:, 1])
        if plot:
            plt.plot(base[:, 1])
            plt.plot(shifted[:, 1])
            plt.savefig(os.path.join(out, str(index), f'shifting{index}_{secondI}.png'), dpi=300)
            plt.close()

        # plt.plot(np.fft.rfftfreq(len(base))[:20], np.abs(baseFFT)[:20])
        # plt.plot(np.fft.rfftfreq(len(shifted))[:20], np.abs(shFFT)[:20])
        # plt.savefig(os.path.join(out, str(index), f'FFTampl{index}_{secondI}.png'), dpi=300)
        # plt.close()
        # plt.plot(np.fft.rfftfreq(len(base))[:20], np.angle(baseFFT)[:20])
        # plt.plot(np.fft.rfftfreq(len(shifted))[:20], np.angle(shFFT)[:20])
        # plt.savefig(os.path.join(out, str(index), f'FFTphase{index}_{secondI}.png'), dpi=300)
        # plt.close()

        delta = ((np.angle(shFFT[maxk]) - np.angle(baseFFT[maxk]) + np.pi)%(2*np.pi)-np.pi) * len(base) / maxk / 2 / np.pi
        deltasFFT.append(delta)
        corr = np.correlate(base[:, 1], shifted[:, 1], mode='full')
        deltasCorr.append(corr.argmax()-len(base))
        if plot:
            plt.plot(np.arange(-1000, 1000), corr[len(base)-1000:len(base)+1000])
            plt.savefig(os.path.join(out, str(index), f'corr{index}_{secondI}.png'), dpi=300)
            plt.close()

    outsheet.cell(row = row_s-1, column=3+index*out_table_column_width).value = 'FFT сдвиг'
    j = 0
    for cell, in outsheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=3+index*out_table_column_width, max_col=3+index*out_table_column_width):
        cell.value = deltasFFT[j]
        j +=1
    outsheet.cell(row = row_s-1, column=4+index*out_table_column_width).value = 'Corr сдвиг'
    j = 0
    for cell, in outsheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=4+index*out_table_column_width, max_col=4+index*out_table_column_width):
        cell.value = deltasCorr[j]
        j +=1


    index += 1
    i+=2




In [4]:
outsheet.cell(row=2, column=3).value
# wb.save(f"{out}/shifts3.xlsx")

'FFT сдвиг'

In [5]:
outwb.save(outtable_name)

In [6]:
    # определение периода
index = 0
i = 0
while i < 70:
    i+=1
    if sheet.cell(row=1, column=i).value == None:
        continue

    # основной код обработки столбика из таблицы
    row_s = 3
    row_end = row_s
    while sheet.cell(row=row_end, column=i).value is not None:
        row_end+=1

    table = pd.DataFrame({
        'сила, кН':
          [i for i, in sheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=i, max_col=i,  values_only=True)],
        'file_name':
          ['WFM' + (str(i) if len(str(i)) > 1 else '0'+str(i))+'.csv' for i, in
               sheet.iter_rows(min_row=row_s, max_row=row_end-1, min_col=i+1, max_col=i+1,  values_only=True)]
    })

    deltasFFT = [0]
    deltasCorr = [0]
    base = np.loadtxt(os.path.join(src, table['file_name'][0]), skiprows=1, delimiter=',')
    base[:, 1] -= base[:, 1].mean()
    baseFFT = np.fft.rfft(base[:, 1])
    maxk = np.abs(baseFFT).argmax()
    print(index, 1/np.fft.rfftfreq(base.shape[0])[maxk])


    index += 1
    i+=2






IndentationError: expected an indented block (<ipython-input-6-a0a4d9c53d0e>, line 5)